In [1]:
import re
import json
from uuid import uuid4
from collections import defaultdict

def m3u_to_monplayer_json(m3u_file_path, provider_name="Tung IPTV", provider_logo=None):
    with open(m3u_file_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]

    # Initialize provider JSON
    provider_id = str(uuid4())
    json_data = {
        "id": provider_id,
        "name": provider_name,
        "description": f"A curated list of channels from {provider_name}",
        "url": provider_name.lower().replace(" ", "_") + ".json",
        "color": "#66CDAA",
        "image": {
            "url": provider_logo or "",
            "type": "contain",
            "height": 1000,
            "width": 1000
        },
        "grid_number": 2,
        "groups": []
    }

    # Group channels by group-title
    groups_dict = defaultdict(list)

    extinf_pattern = re.compile(
        r'#EXTINF:-1(?:\s+tvg-logo="(?P<logo>.*?)")?(?:\s+group-title="(?P<group>.*?)")?,(?P<name>.*)'
    )

    for i, line in enumerate(lines):
        if line.startswith("#EXTINF"):
            match = extinf_pattern.match(line)
            if match:
                logo = match.group("logo") or ""
                group_name = match.group("group") or "Others"
                channel_name = match.group("name")
                url = lines[i + 1] if i + 1 < len(lines) else ""

                channel_id = channel_name.lower().replace(" ", "-")
                stream_id = channel_id + "-full"
                server_id = f"{channel_id}-1-s1"

                channel_json = {
                    "id": channel_id,
                    "name": channel_name,
                    "description": "",
                    "label": "Full",
                    "image": {
                        "url": logo,
                        "type": "cover",
                        "width": 410,
                        "height": 590
                    },
                    "display": "default",
                    "type": "single",
                    "enable_detail": True,
                    "sources": [
                        {
                            "id": str(uuid4()),
                            "name": "Source 1",
                            "image": None,
                            "contents": [
                                {
                                    "id": str(uuid4()),
                                    "name": "Content 1",
                                    "image": None,
                                    "grid_number": 3,
                                    "streams": [
                                        {
                                            "id": stream_id,
                                            "name": channel_name,
                                            "image": {
                                                "url": logo,
                                                "type": "cover",
                                                "width": 410,
                                                "height": 590
                                            },
                                            "stream_links": [
                                                {
                                                    "id": server_id,
                                                    "name": "Server 1",
                                                    "url": url,
                                                    "type": "hls",
                                                    "default": True,
                                                    "enableP2P": False,
                                                    "subtitles": None,
                                                    "remote_data": None,
                                                    "request_headers": None,
                                                    "comments": None
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ],
                            "remote_data": None
                        }
                    ]
                }

                groups_dict[group_name].append(channel_json)

    # Convert groups dict to list
    for group_name, channels in groups_dict.items():
        group_json = {
            "id": f"group-{str(uuid4())}",
            "name": group_name,
            "display": "vertical",
            "image": {
                "url": "",
                "type": "cover",
                "width": 410,
                "height": 590
            },
            "grid_number": 3,
            "enable_detail": False,
            "channels": channels
        }
        json_data["groups"].append(group_json)

    return json_data

# Example usage
m3u_file = "tung_iptv.m3u"
json_result = m3u_to_monplayer_json(
    m3u_file,
    provider_name="Tung IPTV",
    provider_logo="https://raw.githubusercontent.com/lamtung16/iptv/refs/heads/main/images/logo.png"
)

# Save to file
with open("tung_iptv.json", "w", encoding="utf-8") as f:
    json.dump(json_result, f, ensure_ascii=False, indent=2)

print("M3U converted to Monplayer JSON successfully!")

M3U converted to Monplayer JSON successfully!
